In [3]:
# Original coding process
from langchain.document_loaders import PyPDFLoader

# File path for the RBI documents
pdf_paths = [
    "/Users/diboshbaruah/Capstone_Project - Generative AI/financial risk.pdf",
    "/Users/diboshbaruah/Capstone_Project - Generative AI/operations risk.pdf"
]

documents = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())


In [5]:
# Initializing the RecursiveCharacterTextSplitter to break down text into smaller chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitting the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [27]:
# Generating embeddings for each document chunk using DistilBERT
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np

# Loading the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Converting the documents to embeddings directly
doc_embeddings = []

for doc in docs:
    # Tokenizing the document text
    inputs = tokenizer(doc.page_content, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Getting the model outputs without calculating gradients
    with torch.no_grad():
        outputs = model(**inputs)

    # Getting the mean of the last hidden state (this gives us the embedding)
    doc_embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

# Converting the list of embeddings to a NumPy array
doc_embeddings_np = np.array(doc_embeddings)


In [7]:
# Using Hugging Face’s all-MiniLM-L6-v2 to create embeddings for document content
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Initializing the HuggingFaceEmbeddings model with a sentence-transformers model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Sample documents
docs = [
    Document(page_content="This is a financial risk management document."),
    Document(page_content="This is an operational risk management document."),
]

# Creating a FAISS vector store with the documents and embeddings model
vector_store = FAISS.from_documents(docs, embedding_model)

# Saving the FAISS index 
vector_store.save_local("faiss_index")


In [11]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Extract context from document chunks
context = " ".join([doc.page_content for doc in docs])  # Combine all chunks into a single context

max_context_length = 1000

if len(context.split()) > max_context_length:
    summarized_context = summarizer(context, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
else:
    summarized_context = context

# 6. Question-Answering Pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

query = "How loans are processed?"

answer = qa_pipeline(question=query, context=summarized_context)

# Print the Answer
print(answer)


Device set to use mps:0
Device set to use mps:0


{'score': 0.14525197446346283, 'start': 10, 'end': 93, 'answer': 'financial risk management document. This is an operational risk management document'}


In [13]:
from fastapi import FastAPI
from pydantic import BaseModel

# Creating the FastAPI app
app = FastAPI()

# Defining a Pydantic model for the query request
class QueryRequest(BaseModel):
    question: str


In [15]:
import nest_asyncio
import threading
import uvicorn
import logging
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import

# Applying nest_asyncio to allow FastAPI to run on Jupyter
nest_asyncio.apply()

# Suppress all logs at the INFO level and below for relevant loggers
logging.basicConfig(level=logging.WARNING)

# Suppress Uvicorn logs
logging.getLogger("uvicorn").setLevel(logging.WARNING)
logging.getLogger("uvicorn.error").setLevel(logging.WARNING)
logging.getLogger("uvicorn.access").setLevel(logging.WARNING)

# Suppress any other third-party library logs (e.g., FAISS, Hugging Face transformers)
logging.getLogger("transformers").setLevel(logging.WARNING)  # Example for Hugging Face
logging.getLogger("faiss").setLevel(logging.WARNING)  # Example for FAISS

# Initializing the Hugging Face QA pipeline with a fine-tuned model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

# Creating an embedding object using HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Loading the FAISS vector store (Make sure the path is correct where your index is saved)
vector_store = FAISS.load_local("./faiss_index", embedding_model, allow_dangerous_deserialization=True)

# Creating FastAPI app
app = FastAPI()

# Defining a Pydantic model for the query request
class QueryRequest(BaseModel):
    question: str

# Defining the FastAPI endpoint for question answering
@app.post("/ask-question/")
async def ask_question(query_request: QueryRequest):
    query = query_request.question

    # Performing similarity search on the FAISS vector store 
    search_results = vector_store.similarity_search(query, k=1)  

    # Only taking the top chunk as context
    context = search_results[0].page_content  
    
    # Using the QA pipeline to generate an answer based on the context
    answer = qa_pipeline(question=query, context=context)

    # Returning the answer in the response
    return {"answer": answer['answer']}


# Function to run the FastAPI app in a separate thread
def run_fastapi():
    uvicorn.run(app, host="127.0.0.1", port=8007, log_level="info")

# Running the FastAPI app in a separate thread
thread = threading.Thread(target=run_fastapi)
thread.start()


Device set to use mps:0


In [19]:
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loading documents and create embeddings
pdf_paths = ["/Users/diboshbaruah/Capstone_Project - Generative AI/financial risk.pdf",
    "/Users/diboshbaruah/Capstone_Project - Generative AI/operations risk.pdf"]
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

# Splitting documents into smaller chunks for better similarity matching
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Using HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Creating the FAISS vector store
vector_store = FAISS.from_documents(chunks, embedding_model)

# Saving the FAISS index to disk
vector_store.save_local("./faiss_index")


In [21]:
# Sending HTTP POST requests to the FastAPI server

import requests

# The URL of the FastAPI server
url = "http://127.0.0.1:8007/ask-question/"

# Defining the question payload
question_payload = {
    "question": "How loans are processed?"
}

# Sending a POST request to the FastAPI server
response = requests.post(url, json=question_payload)

print()

# Checking the response status code and output the answer
if response.status_code == 200:
    answer = response.json()  
    print(f"Answer: {answer['answer']}")
else:
    print(f"Error: {response.status_code}")


INFO:     127.0.0.1:64363 - "POST /ask-question/ HTTP/1.1" 200 OK

Answer: Loan Review Mechanism


** Now running the saved scripts on jupyter notebook - embedding.py // app.py // client.py ***

In [23]:
# main.py

# Running embedding.py
%run embedding.py

# Running app.py to start the FastAPI server
%run app.py

# Running client.py to test the FastAPI API call
%run client.py


Embedding and FAISS index creation completed.


Device set to use mps:0


FastAPI server started.


INFO:     Started server process [36620]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8007): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:64385 - "POST /ask-question/ HTTP/1.1" 200 OK

Answer: Loan Review Mechanism
